# Named Entity Recognition Using BERT

### Required packages
* pytorch-pretrained-bert
* pandas
* seqeval
* unicode

In [31]:
import sys
import os
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
import random

from pytorch_pretrained_bert.tokenization import BertTokenizer
from torch.optim import Adam

bert_utils_path = os.path.abspath('../../utils_nlp/bert')
if bert_utils_path not in sys.path:
    sys.path.insert(0, bert_utils_path)

from configs import (PathConfig,
                     GlobalConfig, 
                     DeviceConfig, 
                     ModelConfig, 
                     OptimizerConfig, 
                     TrainConfig, 
                     EvalConfig)
from bert_data_utils import KaggleNERProcessor
from bert_utils import (convert_examples_to_token_features,
                        create_train_dataloader, 
                        create_eval_dataloader, 
                        load_model, 
                        get_optimizer_params, 
                        train_model, 
                        eval_token_model)

## Configurations

### Path configuration

In [2]:
path_config_dict = {"data_dir": "./data/NER/", 
                    "output_dir": "./NER_output/"}
path_config = PathConfig(path_config_dict)

### Global configuration

In [3]:
global_config_dict = {"fp16": False}
global_config = GlobalConfig(global_config_dict)

### Device configuration

In [4]:
device_config_dict = {"no_cuda": False}
device_config = DeviceConfig(device_config_dict)
print("device name: {}".format(torch.cuda.get_device_name(0)))
print("number of gpus: {}".format(device_config.n_gpu))

device name: Tesla K80
number of gpus: 1


### Model configuration

In [5]:
model_config_dict = {"bert_model": "bert-base-uncased",
                     "max_seq_length": 75,
                     "num_labels": 18,
                     "model_type": "token"}
model_config = ModelConfig(model_config_dict)

### Optimizer configuration

In [6]:
optimizer_config_dict = {"no_decay_params": ['bias', 'gamma', 'beta'],
                         "learning_rate": 3e-5}
optimizer_config = OptimizerConfig(optimizer_config_dict)

### Train configuration

In [7]:
train_config_dict = {"train_batch_size": 32,
                     "num_train_epochs": 2, 
                     "clip_gradient": True}
train_config = TrainConfig(train_config_dict)

### Evaluation configuration

In [8]:
eval_config = EvalConfig({"eval_batch_size":32})

### Set random seeds

In [9]:
random.seed(global_config.seed)
np.random.seed(global_config.seed)
torch.manual_seed(global_config.seed)

## Preprocess Data

### Create training and validation examples
KaggleNERProcessor is a dataset specific class that generates training and evaluation examples in the format accepted by all utility functions. 

In [10]:
kaggle_ner_processor = KaggleNERProcessor(data_dir="./data/NER/ner_dataset.csv", dev_percentage = 0.1)

In [11]:
train_examples = kaggle_ner_processor.get_train_examples(data_dir="./data/NER/ner_dataset.csv")
dev_examples = kaggle_ner_processor.get_dev_examples(data_dir="./data/NER/ner_dataset.csv")
label_list = kaggle_ner_processor.get_labels()

In [12]:
print('Sample sentence: \n{}\n'.format(train_examples[0].text_a))
print('Sample sentence labels: \n{}\n'.format(train_examples[1].label))

Sample sentence: 
Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .

Sample sentence labels: 
['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-tim', 'O', 'O', 'O', 'B-org', 'O', 'O', 'O', 'O', 'O']



### Convert examples to features
The function `convert_examples_to_token_features` converts raw string data to numerical features, involving the following steps:
1. Tokenization
2. Convert tokens and labels to numerical values
3. Sequence padding or truncation

In [13]:
tokenizer = BertTokenizer.from_pretrained(model_config.bert_model,
                                          do_lower_case=model_config.do_lower_case)
train_features = convert_examples_to_token_features(examples=train_examples,
                                                    tokenizer=tokenizer,
                                                    label_list=label_list, 
                                                    model_config=model_config)
dev_features = convert_examples_to_token_features(examples=dev_examples,
                                                  tokenizer=tokenizer,
                                                  label_list=label_list, 
                                                  model_config=model_config)

In [14]:
print("Sample token id:\n{}\n".format(train_features[0].input_ids))
print("Sample attention mask:\n{}\n".format(train_features[0].input_mask))
print("Sample label ids:\n{}\n".format(train_features[0].label_id))

Sample token id:
[5190, 1997, 28337, 2031, 9847, 2083, 2414, 2000, 6186, 1996, 2162, 1999, 5712, 1998, 5157, 1996, 10534, 1997, 2329, 3629, 2013, 2008, 2406, 1012, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Sample attention mask:
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Sample label ids:
[1, 1, 1, 1, 1, 1, 13, 1, 1, 1, 1, 1, 13, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

### Create dataloaders
The utility functions `create_train_dataloader` and `create_eval_dataloader` creates Pytorch dataloaders from features, which can be used for model training and evaluation. The following two steps are performed:
1. Convert numpy arrays to Pytorch tensors
2. Create dataloader for sampling and serving data in batches

In [15]:
train_dataloader = create_train_dataloader(train_features=train_features,
                                           model_config=model_config,
                                           train_config=train_config,
                                           device_config=device_config)

In [16]:
valid_dataloader = create_eval_dataloader(eval_features=dev_features, 
                                          model_config=model_config, 
                                          eval_config=eval_config)

In [17]:
it = iter(train_dataloader)
first = next(it)
print("Sample token id tensor:\n{}\n".format(first[0][0]))
print("Sample attention mask tensor:\n{}\n".format(first[1][0]))
print("Sample label id tensor:\n{}\n".format(first[3][0]))

Sample token id tensor:
tensor([ 1057, 29625,  2015, 29625, 29624,  3709,  2749,  1999,  7041,  2360,
         2027,  2730,  2321, 17671,  2076,  2019, 11585,  3169,  1999,  1996,
         2264,  1010,  2096,  2334,  4584,  2758,  2216,  2730,  2020,  9272,
         1012,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0])

Sample attention mask tensor:
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0])

Sample label id tensor:
tensor([ 1, 17, 17, 17, 17, 17,  1,  1, 13,  1,  1,  1,  1,  1,  1,  1, 

## Load Model

In [18]:
model = load_model(model_config=model_config, 
                   path_config=path_config, 
                   device_config=device_config,
                   global_config=global_config)

## Configure Optimizer
This step must be done after loading the model, because the load_model function moves all model parameters to the device, e.g. GPU. 

In [19]:
optimizer_config = get_optimizer_params(optimizer_config=optimizer_config,
                                        train_config=train_config, 
                                        device_config=device_config, 
                                        model=model, 
                                        num_train_examples=len(train_dataloader))

In [20]:
# optimizer = BertAdam(optimizer_grouped_parameters,
#                      lr=optimizer_config.learning_rate,
#                      warmup=optimizer_config.warmup_proportion,
#                      t_total=num_train_optimization_steps)
optimizer = Adam(optimizer_config.grouped_parameters, lr=optimizer_config.learning_rate)

## Train Model

In [21]:
model, train_loss = train_model(model=model, 
                                train_dataloader=train_dataloader, 
                                optimizer=optimizer,
                                train_config=train_config, 
                                model_config=model_config, 
                                optimizer_config=optimizer_config,
                                device_config=device_config,
                                global_config=global_config)

Iteration:   9%|▉         | 126/1349 [01:57<18:59,  1.07it/s]


Iteration:  19%|█▊        | 252/1349 [03:55<17:08,  1.07it/s]


Iteration:  28%|██▊       | 378/1349 [05:54<15:14,  1.06it/s]


Iteration:  37%|███▋      | 504/1349 [07:53<13:19,  1.06it/s]


Iteration:  47%|████▋     | 630/1349 [09:52<11:25,  1.05it/s]


...

Iteration:  56%|█████▌    | 752/1349 [11:59<09:34,  1.04it/s]


Iteration:  65%|██████▌   | 878/1349 [14:00<07:30,  1.04it/s]


Iteration:  74%|███████▍  | 1003/1349 [16:00<05:31,  1.04it/s]


Iteration:  84%|████████▎ | 1127/1349 [17:58<03:32,  1.04it/s]


Iteration:  91%|█████████▏| 1234/1349 [19:41<01:50,  1.04it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Evaluate Model

In [22]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [23]:
preds, eval_loss, eval_accuracy = eval_token_model(model=model, 
                                                   eval_dataloader=valid_dataloader, 
                                                   model_config=model_config, 
                                                   device_config=device_config, 
                                                   label_list=label_list,
                                                   eval_func=flat_accuracy)

Validation loss: 0.2494839150706927
Validation Accuracy: 0.9776226190476196
Validation F1-Score: 0.7582687879422232
